In [65]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import sys
import os
import torch
import warnings
import yaml
from easydict import EasyDict as edict
from pprint import pprint

%load_ext autoreload
%autoreload 2
%matplotlib inline

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (16, 3)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
import yfinance as yf
from pandas_datareader import data as pdr

yf.pdr_override()

from datetime import datetime as dt

import yaml


class StockAnalyer:
    def __init__(self, cfg_name="cfg-quant", cfg_dir="./configs"):
        with open(f"{cfg_dir}/{cfg_name}.yaml") as file:
            cfg = yaml.load(file, Loader=yaml.FullLoader)
            self.cfg = edict(cfg)
        self.__load__()
        self.__set__()

    def __load__(self):
        tickers = self.cfg.tickers

        data_dic = {}
        for key, ticker in tickers.items():
            data_dic[key] = yf.Ticker(ticker)

        self.data_dic = data_dic

    def __set__(self):
        company_name = "asml"
        comp = self.data_dic[company_name]

        info = comp.info
        bal = comp.balance_sheet
        inc = comp.income_stmt
        fin = comp.financials
        cash = comp.cashflow

        # info
        beta = info["beta"]
        market_cap = info["marketCap"]

        # financials
        ebit = fin.loc["EBIT"][0]
        ebitda = fin.loc["EBITDA"][0]

        # balance
        total_debt = bal.loc["Total Debt"][0]
        total_asset = bal.loc["Total Assets"][0]

        # cash
        total_cash = cash.loc["Free Cash Flow"][0]

        # * Statistics

        # good company
        nopat = inc.loc["Net Income"][0]
        net_tangible_assets = bal.loc["Net Tangible Assets"][0]
        non_current_asset = bal.loc["Total Non Current Assets"][0]
        working_capital = bal.loc["Working Capital"][0]
        modified_roic = nopat / (non_current_asset + working_capital)
        print(f"modified_roic: {modified_roic:.2f}")

        # good earning
        ev = market_cap - total_debt + total_cash
        earning_rate = ebit / ev
        print(f"earning_rate: {earning_rate:.2f}")

        # good quality(gross profitability)
        total_revenue = inc.loc["Total Revenue"][0]
        gross_profit = inc.loc["Gross Profit"][0]

        gpa = gross_profit / (total_asset - total_debt)
        print(f"gpa: {gpa:.2f}")

        # good price PER, PBR, 1/PBR = BM

    def get_data_dic(self):
        return self.data_dic

    def forward(self):
        pass


stock_analyer = StockAnalyer()

modified_roic: 0.33
earning_rate: 0.02
gpa: 0.40
